# MSSQLDB Manager Usage Examples

This notebook demonstrates how to use the `MSSQLDB` manager class to create, manage, and interact with a Microsoft SQL Server database in a Docker container. We'll use Python and SQL cell magic to showcase the package functionality.

## Setup

### Install Required Packages

In [1]:
!pip install ipython-sqlcmd py-dockerdb pyodbc


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import Dependencies

In [2]:
import uuid
from pathlib import Path
from docker_db.mssql_db import MSSQLConfig, MSSQLDB

# For SQL cell magic
%load_ext sqlcmd

## Creating a SQL Server Database Instance

Let's create a temporary directory for our database files:

In [3]:
import tempfile
import os
temp_dir = Path("tmp").absolute()
temp_dir.mkdir(exist_ok=True)
container_name = f"demo-mssql-{uuid.uuid4().hex[:8]}"
init_script_path = Path("configs", "mssql", "initdb.sql")
init_script_path.exists()

True

In [4]:
from utils import display_sql_script
display_sql_script(init_script_path)

```sql
-- MSSQL Initialization Script
-- This script runs when the database container starts

-- Ensure we're using the correct database
USE demodb;

-- Create table if it doesn't exist
IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='test_table' AND xtype='U')
CREATE TABLE test_table (
    id INT IDENTITY(1,1) PRIMARY KEY,
    name NVARCHAR(255),
    environment NVARCHAR(100)
);

-- Insert test data using environment variable (passed via environment)
-- Note: MSSQL doesn't support environment variables directly in SQL like MySQL
-- So we'll handle this through the application layer
INSERT INTO test_table (name, environment) 
VALUES ('Test Entry', '$(YourEnvVar)');

GO;
-- Create a sample function
CREATE OR ALTER FUNCTION dbo.get_env_info()
RETURNS NVARCHAR(500)
AS
BEGIN
    DECLARE @env_value NVARCHAR(255);
    SELECT TOP 1 @env_value = environment FROM test_table;
    RETURN 'Environment: ' + ISNULL(@env_value, 'Not Set');
END;
GO;
```

Now, let's set up the MSSQLDB configuration:

In [5]:
# Generate a unique container name
container_name = f"demo-mssql-{uuid.uuid4().hex[:8]}"

# Create a configuration for our database
config = MSSQLConfig(
    user="demouser",
    host="127.0.0.1",               # <- for some reason, this is not working with 'localhost'
    password="Demo_Pass123",
    database="demodb",
    sa_password="StrongPass123!",
    project_name="demo",
    workdir=temp_dir,
    container_name=container_name,
    retries=20,
    delay=3,
    init_script=init_script_path,
    env_vars={"YourEnvVar": "TestEnvironment"}
)

# Initialize the database manager
db_manager = MSSQLDB(config)

## Start the Database

We'll now create and start the database. This process may take a bit longer for SQL Server as it's a more resource-intensive database engine:

In [6]:
# Create and start the database
db_manager.create_db()
print(f"Database started successfully in container '{container_name}'")
print(f"Connection details:")
print(f"  Host: {config.host}")
print(f"  Port: {config.port}")
print(f"  User: {config.user}")
print(f"  Database: {config.database}")

DB unreachable. Obtained error: ('28000', "[28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user 'demouser'. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 17 for SQL Server]Invalid connection string attribute (0); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user 'demouser'. (18456); [28000] [Microsoft][ODBC Driver 17 for SQL Server]Invalid connection string attribute (0)")


InterfaceError: ('28000', "[28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user 'demouser'. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 17 for SQL Server]Invalid connection string attribute (0); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user 'demouser'. (18456); [28000] [Microsoft][ODBC Driver 17 for SQL Server]Invalid connection string attribute (0)")

In [ ]:
import time
import pyodbc

# Connect as sa first to create the demouser
sa_conn_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={config.host};DATABASE=master;UID=sa;PWD={config.sa_password};Encrypt=yes;TrustServerCertificate=yes;"

try:
    # Connect as sa to create user and database
    sa_conn = pyodbc.connect(sa_conn_string)
    cursor = sa_conn.cursor()
    
    # Create the database
    cursor.execute(f"CREATE DATABASE {config.database}")
    print(f"Database '{config.database}' created successfully")
    
    # Create the login and user
    cursor.execute(f"CREATE LOGIN {config.user} WITH PASSWORD = '{config.password}'")
    cursor.execute(f"USE {config.database}")
    cursor.execute(f"CREATE USER {config.user} FOR LOGIN {config.user}")
    cursor.execute(f"ALTER ROLE db_owner ADD MEMBER {config.user}")
    
    print(f"User '{config.user}' created successfully with db_owner permissions")
    
    cursor.close()
    sa_conn.close()
    
except Exception as e:
    print(f"Error during setup: {e}")

print(f"Database started successfully in container '{container_name}'")
print(f"Connection details:")
print(f"  Host: {config.host}")
print(f"  Port: {config.port}")
print(f"  User: {config.user}")
print(f"  Database: {config.database}")

Waiting for SQL Server to be fully ready...
Error during setup: ('42000', '[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]CREATE DATABASE statement not allowed within multi-statement transaction. (226) (SQLExecDirectW)')
Database started successfully in container 'demo-mssql-5a19293b'
Connection details:
  Host: 127.0.0.1
  Port: 1433
  User: demouser
  Database: demodb


## Connect and Run SQL Queries

Now that our database is running, let's connect to it using SQL cell magic. For SQL Server, we'll use the ODBC connection string:

In [ ]:
%sqlcmd $config.database --password $config.password --username $config.user --driver "ODBC Driver 17 for SQL Server"  --server $config.host

Connection string set: mssql+sqlcmd:///?odbc_connect=DRIVER%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BSERVER%3D127.0.0.1%3BDATABASE%3Ddemodb%3BUID%3Ddemouser%3BPWD%3DDemo_Pass123%3BEncrypt%3Dyes%3BTrustServerCertificate%3Dyes%3B
Parsed connection info: {'server': '127.0.0.1', 'database': 'demodb', 'username': 'demouser', 'password': '***'}


### Creating a Table

In [ ]:
%%sqlcmd
CREATE TABLE demo_users (
    id INT IDENTITY(1,1) PRIMARY KEY,
    username VARCHAR(50) UNIQUE NOT NULL,
    email VARCHAR(100) UNIQUE,
    created_at DATETIME DEFAULT GETDATE()
);

### Insert Data

In [ ]:
%%sqlcmd
INSERT INTO demo_users (username, email) VALUES
    ('alice', 'alice@example.com'),
    ('bob', 'bob@example.com'),
    ('charlie', 'charlie@example.com');

(3 rows affected)


### Query Data

In [ ]:
%%sqlcmd
SELECT * FROM demo_users;

,id,username,email,created_at
0,1,alice,alice@example.com,2025-05-20 08:52:15.097
1,2,bob,bob@example.com,2025-05-20 08:52:15.097
2,3,charlie,charlie@example.com,2025-05-20 08:52:15.097



(3 rows affected)


### Run More Complex Queries

In [ ]:
%%sqlcmd
-- Create another table for demonstration
CREATE TABLE demo_posts (
    id INT IDENTITY(1,1) PRIMARY KEY,
    user_id INT FOREIGN KEY REFERENCES demo_users(id),
    title VARCHAR(100) NOT NULL,
    content NVARCHAR(MAX),
    created_at DATETIME DEFAULT GETDATE()
);

-- Insert some posts
INSERT INTO demo_posts (user_id, title, content) VALUES
    (1, 'Alice First Post', 'Hello world from Alice!'),
    (1, 'Alice Second Post', 'Another post from Alice'),
    (2, 'Bob Introduction', 'Hi, this is Bob!'),
    (3, 'Charlie Notes', 'Some notes from Charlie');
    
-- Query with JOIN
SELECT u.username, p.title, p.content
FROM demo_users u
JOIN demo_posts p ON u.id = p.user_id
ORDER BY u.username, p.created_at;

(4 rows affected)


,username,title,content
0,alice,Alice First Post,Hello world from Alice!
1,alice,Alice Second Post,Another post from Alice
2,bob,Bob Introduction,"Hi, this is Bob!"
3,charlie,Charlie Notes,Some notes from Charlie



(4 rows affected)


## Using Regular Python to Access the Database

You can also interact with the database using Python code and pyodbc:

In [ ]:
# Connect directly using our db_manager's connection property
conn = db_manager.connection

# Create a cursor
cursor = conn.cursor()

# Execute a query
cursor.execute("SELECT COUNT(*) as post_count FROM demo_posts")
result = cursor.fetchone()
print(f"Total number of posts: {result[0]}")

# Group by query
cursor.execute("""
    SELECT u.username, COUNT(p.id) as post_count 
    FROM demo_users u
    LEFT JOIN demo_posts p ON u.id = p.user_id
    GROUP BY u.username
    ORDER BY post_count DESC
""")

print("\nPost count by user:")
for row in cursor.fetchall():
    print(f"  {row[0]}: {row[1]} posts")

# Close the connection
cursor.close()
conn.close()

Total number of posts: 4

Post count by user:
  alice: 2 posts
  bob: 1 posts
  charlie: 1 posts


## Using SQL Server-Specific Features

Let's try some SQL Server-specific features like T-SQL stored procedures:

In [ ]:
%%sqlcmd
-- Create a stored procedure to get post count for a specific user
CREATE PROCEDURE GetUserPostCount
    @username VARCHAR(50)
AS
BEGIN
    SELECT u.username, COUNT(p.id) as post_count
    FROM demo_users u
    LEFT JOIN demo_posts p ON u.id = p.user_id
    WHERE u.username = @username
    GROUP BY u.username
END;

-- Execute the stored procedure
EXEC GetUserPostCount @username = 'alice';

## Clean Up

When you're done with the database, you can delete it:

In [ ]:
# Delete the database container
db_manager.delete_db()
print(f"Database container '{container_name}' deleted")

# Clean up the temporary directory
import shutil
shutil.rmtree(temp_dir)
print(f"Temporary directory '{temp_dir}' removed")

Database container 'demo-mssql-3ab9f308' deleted
Temporary directory 'C:\Users\acisse\AppData\Local\Temp\tmp0twu9uin' removed


## Conclusion

This notebook demonstrated how to:

1. Configure and create a Microsoft SQL Server database with `MSSQLDB`
2. Connect to the database using SQL cell magic with pyodbc
3. Execute SQL queries and create T-SQL stored procedures
4. Use pyodbc with Python to interact with the database
5. Clean up the database when finished

The `MSSQLDB` manager provides a convenient way to spin up SQL Server instances in Docker containers for development, testing, or demonstration purposes.

Note that SQL Server requires more resources than some other database engines, so ensure your Docker environment has sufficient memory and CPU allocated.